# Initial data cleaning - Year 2017

[Columns explanation](https://aqs.epa.gov/aqsweb/airdata/FileFormats.html#_hourly_data_files)

[Data source](https://aqs.epa.gov/aqsweb/airdata/download_files.html#Raw)

### Goals:
* identify and unimportant columns
* shorten data set, find most densly populated states, counties, sites
* create unique identifier for each metering site *SiteCode*
* save to parquet

### TODO:
* save WIND to parquet
* do the same for TEMP, PRESS, RH_DP and PM2.5 FRM/FEM Mass, PM2.5 non FRM/FEM Mass, PM10 Mass, PM2.5 Speciation, PM10 Speciation 

In [1]:
from os.path import join, basename, splitext
from glob import glob
from dask import dataframe as dd
from matplotlib import rcParams
import pandas as pd
import dask


from deep_aqi import ROOT


pd.set_option('max_columns', 50)
pd.set_option('max_rows', 25)

In [2]:
RAW_DATA = join(ROOT, 'data', 'raw')
INTERIM_DATA = join(ROOT, 'data', 'interim')

In [3]:
files = glob(f'{RAW_DATA}/*.csv', recursive=True)
files

['/home/filip/projects/deep_aqi/deep_aqi/data/raw/hourly_WIND_2017.csv',
 '/home/filip/projects/deep_aqi/deep_aqi/data/raw/hourly_PRESS_2017.csv',
 '/home/filip/projects/deep_aqi/deep_aqi/data/raw/hourly_88101_2017.csv',
 '/home/filip/projects/deep_aqi/deep_aqi/data/raw/hourly_TEMP_2017.csv']

### WIND

In [4]:
# there was some inconsistency with 'Qualifier' column
data = dd.read_csv(files[2], dtype={'Qualifier': 'object'})

In [5]:
data.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 24 entries, State Code to Date of Last Change
dtypes: object(13), float64(4), int64(7)

In [6]:
data.head()

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Date Local,Time Local,Date GMT,Time GMT,Sample Measurement,Units of Measure,MDL,Uncertainty,Qualifier,Method Type,Method Code,Method Name,State Name,County Name,Date of Last Change
0,1,73,23,88101,3,33.553056,-86.815,WGS84,PM2.5 - Local Conditions,2017-01-01,00:00,2017-01-01,06:00,2.3,Micrograms/cubic meter (LC),2,NaN,NaN,FEM,183,Thermo Scientific 5014i or FH62C14-DHS w/VSCC ...,Alabama,Jefferson,2017-04-19
1,1,73,23,88101,3,33.553056,-86.815,WGS84,PM2.5 - Local Conditions,2017-01-01,01:00,2017-01-01,07:00,9.0,Micrograms/cubic meter (LC),2,NaN,NaN,FEM,183,Thermo Scientific 5014i or FH62C14-DHS w/VSCC ...,Alabama,Jefferson,2017-04-19
2,1,73,23,88101,3,33.553056,-86.815,WGS84,PM2.5 - Local Conditions,2017-01-01,02:00,2017-01-01,08:00,8.2,Micrograms/cubic meter (LC),2,NaN,NaN,FEM,183,Thermo Scientific 5014i or FH62C14-DHS w/VSCC ...,Alabama,Jefferson,2017-04-19
3,1,73,23,88101,3,33.553056,-86.815,WGS84,PM2.5 - Local Conditions,2017-01-01,03:00,2017-01-01,09:00,3.1,Micrograms/cubic meter (LC),2,NaN,NaN,FEM,183,Thermo Scientific 5014i or FH62C14-DHS w/VSCC ...,Alabama,Jefferson,2017-04-19
4,1,73,23,88101,3,33.553056,-86.815,WGS84,PM2.5 - Local Conditions,2017-01-01,04:00,2017-01-01,10:00,4.0,Micrograms/cubic meter (LC),2,NaN,NaN,FEM,183,Thermo Scientific 5014i or FH62C14-DHS w/VSCC ...,Alabama,Jefferson,2017-04-19


In [25]:
# Site Num is unique only within county, SiteCode is unique across all counties and states
data['SiteCode'] = data['State Name'] + '_' + data['County Name'] + '_' + data['Site Num'].astype(str)

In [26]:
# rcParams['figure.figsize'] = [20, 10]

# data['State Name'].value_counts().compute().plot(kind='bar')

## Dropping columns
[Columns explanation](https://aqs.epa.gov/aqsweb/airdata/FileFormats.html#_hourly_data_files)

Columns dropped:
* State Code, County Code - same information as in State Name, County Name
* State Name, County Name, Site Num - now all concatenated into SiteCode
* Parameter Code - same information as in Parameter Name
* Datum - seemingly unimportant
* MDL - seemingly unimportant
* Uncertainty - seemingly unimportant (mostly empty column)
* Qualifier - seemingly unimportant (mostly empty column)
* Method Type - seemingly unimportant
* Method Code - seemingly unimportant
* Date of Last Change - unimportant
* Date GTM, Time GMT - concatenated into GMTDate, leaving only LocalDate
* Date Local, Time Local - concatenated into LocalDate
* Longitude, Latitude - if needed I'll get it from source 

In [27]:
# KEEP_COLS = ['SiteCode', 'State Name', 'County Name', 'Latitude', 'Longitude', 'LocalDate',
#              'GMTDate', 'Parameter Name', 'POC', 'Sample Measurement',
#              'Units of Measure']

# no unessential columns
KEEP_COLS = ['SiteCode', 'LocalDate', 'Parameter Name', 'POC', 'Sample Measurement', 'Units of Measure']

In [28]:
# site_summary = data.groupby(by=['SiteCode'])['Sample Measurement'].agg('count').compute().sort_values(ascending=False)

In [29]:
data['Local Date'] = data['Date Local'] + ' ' + data['Time Local']
# data['GMT Date'] = data['Date GMT'] + ' ' + data['Time GMT']

data['LocalDate'] = data['Local Date'].map_partitions(pd.to_datetime,
                                                      format='%Y/%m/%d %H:%M',
                                                      meta=('Local Date'))

# data['GMTDate'] = data['GMT Date'].map_partitions(pd.to_datetime,
#                                                   format='%Y/%m/%d %H:%M',
#                                                   meta=('GMT Date'))

In [30]:
data = data[KEEP_COLS]

### Explanation for number of occurrences per site

There are 8756 hours in a year
sites with n ~ 17520 have measurements of both wind speed and wind direction
sites with n > 17520 have measurements made by multiple instruments - take average value from all instruments for each hour

In [9]:
# index_cols = ['SiteCode', 'State Name', 'County Name', 'Latitude', 'Longitude', 'LocalDate']
# index_fields = data[index_cols]

In [10]:
# index_fields = index_fields.drop_duplicates(subset='SiteCode')

In [31]:
values = data.groupby(by=['SiteCode', 'LocalDate', 'Parameter Name'])['Sample Measurement'].mean()

In [12]:
values.head()


SiteCode                LocalDate            Parameter Name            
Alabama_Jefferson_2059  2017-01-01 00:00:00  Wind Direction - Resultant    165.0
                                             Wind Speed - Resultant          0.3
                        2017-01-01 01:00:00  Wind Direction - Resultant    158.0
                                             Wind Speed - Resultant          0.2
                        2017-01-01 02:00:00  Wind Direction - Resultant    113.0
Name: Sample Measurement, dtype: float64

In [32]:
values = values.reset_index().compute()

In [33]:
values.columns

Index(['SiteCode', 'LocalDate', 'Parameter Name', 'Sample Measurement'], dtype='object')

In [34]:
values = values.pivot_table(values='Sample Measurement',
                            index=['SiteCode', 'LocalDate'],
                            columns='Parameter Name')

In [35]:
values = values.reset_index()

In [17]:
values.head()

Parameter Name,SiteCode,LocalDate,Wind Direction - Resultant,Wind Speed - Resultant
0,Alabama_Jefferson_2059,2017-01-01 00:00:00,165.0,0.3
1,Alabama_Jefferson_2059,2017-01-01 01:00:00,158.0,0.2
2,Alabama_Jefferson_2059,2017-01-01 02:00:00,113.0,0.4
3,Alabama_Jefferson_2059,2017-01-01 03:00:00,110.0,0.3
4,Alabama_Jefferson_2059,2017-01-01 04:00:00,63.0,0.1


In [18]:
values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6201309 entries, 0 to 6201308
Data columns (total 4 columns):
SiteCode                      object
LocalDate                     datetime64[ns]
Wind Direction - Resultant    float64
Wind Speed - Resultant        float64
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 189.2+ MB


## 8765 hours in a year; 90% ~ 7888

In [36]:
site_count = values.SiteCode.value_counts()
eligible_sites = site_count[site_count > 7888].index

In [58]:
values = values.loc[values.SiteCode.isin(eligible_sites), :]
values.reset_index(drop=True, inplace=True)

In [59]:
values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5438927 entries, 0 to 5438926
Data columns (total 4 columns):
SiteCode                      object
LocalDate                     datetime64[ns]
Wind Direction - Resultant    float64
Wind Speed - Resultant        float64
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 166.0+ MB


In [60]:
values.head(10)

Parameter Name,SiteCode,LocalDate,Wind Direction - Resultant,Wind Speed - Resultant
0,Alabama_Jefferson_2059,2017-01-01 00:00:00,165.0,0.3
1,Alabama_Jefferson_2059,2017-01-01 01:00:00,158.0,0.2
2,Alabama_Jefferson_2059,2017-01-01 02:00:00,113.0,0.4
3,Alabama_Jefferson_2059,2017-01-01 03:00:00,110.0,0.3
4,Alabama_Jefferson_2059,2017-01-01 04:00:00,63.0,0.1
5,Alabama_Jefferson_2059,2017-01-01 05:00:00,86.0,0.3
6,Alabama_Jefferson_2059,2017-01-01 06:00:00,57.0,0.5
7,Alabama_Jefferson_2059,2017-01-01 07:00:00,63.0,0.6
8,Alabama_Jefferson_2059,2017-01-01 08:00:00,62.0,0.4
9,Alabama_Jefferson_2059,2017-01-01 09:00:00,79.0,0.6


In [61]:
filename = f'{splitext(basename(files[0]))[0]}.parquet'
save_path = join(INTERIM_DATA, filename)

In [62]:
# values.to_parquet(save_path, engine='fastparquet', compression='SNAPPY')